In [11]:
import pandas as pd
import numpy as np
import os 
from tqdm import tqdm

In [12]:

dfs = []
entries = os.scandir('datasets/cleaned_data_version2/extract')
print(type(entries))
for entry in tqdm(entries):
  if entry.is_file():
    path = entry.path
    # print(path)
    df = pd.read_csv(path)
    dfs.append(df)

<class 'nt.ScandirIterator'>


159it [00:02, 68.99it/s]


In [13]:
df.shape

(2393, 4)

In [14]:
df = pd.concat(dfs, ignore_index=True) 

In [15]:
pd.set_option('display.max_colwidth', None)
# df[df['fileID'] == 'C2E001'].iloc[1:40]

In [16]:
df[df['speaker'] == '<PLAYER76>']

,fileID,number,speaker,utterance


## Prepare D&D Info

In [17]:
classdf = pd.read_csv('datasets/dandd_info/classes.csv')
equipdf = pd.read_csv('datasets/dandd_info/equipment.csv')
monstdf = pd.read_csv('datasets/dandd_info/monsters.csv')
spelldf = pd.read_csv('datasets/dandd_info/spells.csv')
racesdf = pd.read_csv('datasets/dandd_info/races.csv')

In [18]:
spell_names = spelldf['name'].tolist()
spell_names

['Acid Arrow',
 'Acid Splash',
 'Aid',
 'Alarm',
 'Alter Self',
 'Animal Friendship',
 'Animal Messenger',
 'Animal Shapes',
 'Animate Dead',
 'Animate Objects',
 'Antilife Shell',
 'Antimagic Field',
 'Antipathy/Sympathy',
 'Arcane Eye',
 'Arcane Hand',
 'Arcane Lock',
 'Arcane Sword',
 "Arcanist's Magic Aura",
 'Astral Projection',
 'Augury',
 'Awaken',
 'Bane',
 'Banishment',
 'Barkskin',
 'Beacon of Hope',
 'Bestow Curse',
 'Black Tentacles',
 'Blade Barrier',
 'Bless',
 'Blight',
 'Blindness/Deafness',
 'Blink',
 'Blur',
 'Branding Smite',
 'Burning Hands',
 'Call Lightning',
 'Calm Emotions',
 'Chain Lightning',
 'Charm Person',
 'Chill Touch',
 'Circle of Death',
 'Clairvoyance',
 'Clone',
 'Cloudkill',
 'Color Spray',
 'Command',
 'Commune',
 'Commune with Nature',
 'Comprehend Languages',
 'Compulsion',
 'Cone of Cold',
 'Confusion',
 'Conjure Animals',
 'Conjure Celestial',
 'Conjure Elemental',
 'Conjure Fey',
 'Conjure Minor Elementals',
 'Conjure Woodland Beings',
 'Contac

## Prepare Critical Role info
 - scraping? -> yes lol

### Actor , Character, Class, Race, ID

In [19]:
import requests
from bs4 import BeautifulSoup

# campaign_1
c1_url = 'https://criticalrole.fandom.com/wiki/Campaign_One:_Vox_Machina'
c1_page = requests.get(c1_url)

c1_soup = BeautifulSoup(c1_page.content, 'html.parser')


In [20]:

def parse_actor_table(table):
    rows = table.find_all('tr')

    # headers
    header_row = rows[0]
    header_cells = header_row.find_all('th')
    header_names = [cell.text.strip() for cell in header_cells]

    # data
    data_rows = rows[1:]
    data_cells = []

    current_actor = ''
    for row in data_rows:
        cells = row.find_all('td')
        if len(cells) != 3:  # new actor
            current_actor = cells[0].text.strip()
            data_cells.append([cell.text.strip() for cell in cells])
        else:
            data_cells.append([current_actor] + [cell.text.strip() for cell in cells])

    return header_names, data_cells


In [21]:
# tables
c1_tables = c1_soup.find_all('table')

c1_header_names, c1_main_data_cells = parse_actor_table(c1_tables[0])
c1_main_df = pd.DataFrame(c1_main_data_cells, columns=c1_header_names)

c1_guests_header_names, c1_guests_data_cells = parse_actor_table(c1_tables[1])
c1_guests_df = pd.DataFrame(c1_guests_data_cells, columns=c1_guests_header_names)


display(c1_main_df)
display(c1_guests_df)


,Actor,Character,Race,Class
0,Orion Acaba(episode 1-27),Tiberius Stormwind,Dragonborn,Sorcerer (Draconic Bloodline)
1,Laura Bailey,Vex'ahlia,Half-elf,Ranger (Beast Master)/Rogue (Assassin)
2,Taliesin Jaffe,Percival de Rolo,Human,Fighter (Gunslinger)
3,Ashley Johnson,Pike Trickfoot,Gnome,Cleric (War Domain)
4,Liam O'Brien,Vax'ildan,Half-elf,Rogue (Assassin)/Paladin (Oath of Vengeance)/Druid
5,Liam O'Brien,Lieve'tel Toluse,Elf,Cleric (Grave Domain)
6,Liam O'Brien,Derrig,Half-elf,Fighter (Battle Master)
7,Marisha Ray,Keyleth,Half-elf,Druid (Circle of the Moon)
8,Sam Riegel,Scanlan Shorthalt,Gnome,Bard (College of Lore)
9,Sam Riegel,Taryon Darrington,Human,Artificer (Battle Smith; formerly Alchemist)


,Actor,Character,Race,Class
0,Mary Elizabeth McGlynn,Zahra Hydris,Tiefling (Bloodline of Asmodeus),"Warlock (The Great Old One, Pact of the Tome)"
1,Felicia Day,Lyra,Human,Wizard (School of Transmutation)
2,Will Friedle,Kashaw Vesh,Human,Cleric (Life Domain)
3,Wil Wheaton,Thorbir Falbek,Dwarf,Fighter (Battle Master)
4,Kit Buss,Lillith Daturai,Tiefling (Bloodline of Asmodeus),Wizard (School of Illusion)
5,Jason Charles Miller,Garthok,Half-orc,Rogue (Thief)
6,Chris Hardwick,Gern Blanston,Dragonborn (Ravenite),Wizard (School of Necromancy)
7,Chris Perkins,Shale,Goliath,Fighter (Battle Master)
8,Patrick Rothfuss,Kerrek,Human,Paladin (Oath of Devotion)
9,ND Stevenson,Tova,Dwarf (Werebear),Blood hunter (Order of the Lycan)


In [22]:
actordict = {'MATT': '<DM>', 'TRAVIS': '<PLAYER1>', 'MARISHA': '<PLAYER2>', 'TALIESIN': '<PLAYER3>', 'SAM': '<PLAYER4>', 'ORION': '<PLAYER5>', 'LIAM': '<PLAYER6>', 'LAURA': '<PLAYER7>', 'ZAC': '<PLAYER8>', 'ALL': '<PLAYER9>', 'ALL - MATT': '<PLAYER10>', 'PRODUCER': '<PLAYER11>', 'CREW': '<PLAYER12>', 'ASHLEY': '<PLAYER13>', 'WIL': '<PLAYER14>', 'IVAN': '<PLAYER15>', 'BECCA': '<PLAYER16>', 'IFY': '<PLAYER17>', 'HECTOR': '<PLAYER18>', 'LUCAS': '<PLAYER19>', 'ALL - TRAVIS - ORION - LIAM - AHSLEY - MATT': '<PLAYER20>', 'COMPUTER': '<PLAYER21>', 'ALL - SAM - MATT': '<PLAYER22>', 'ERIKA': '<PLAYER23>', 'KAI': '<PLAYER24>', 'TAYLOR': '<PLAYER25>', 'JORE': '<PLAYER26>', 'MARY': '<PLAYER27>', 'FELICIA': '<PLAYER28>', 'RYAN': '<PLAYER29>', 'WILL': '<PLAYER30>', 'STEVE': '<PLAYER31>', 'DARIN': '<PLAYER32>', 'KIT': '<PLAYER33>', 'ALL - ORION': '<PLAYER34>', 'ALEC': '<PLAYER35>', 'OFFSCREEN': '<PLAYER36>', 'CHRIS PERKINS': '<PLAYER37>', 'JASON': '<PLAYER38>', 'CHRIS HARDWICK': '<PLAYER39>', 'PATRICK ROTHFUSS': '<PLAYER40>', 'ALL - TALIESIN': '<PLAYER41>', 'SOUND': '<PLAYER42>', 'KEVIN': '<PLAYER43>', 'AUDIENCE': '<PLAYER44>', 'AUDIENCE MEMBER': '<PLAYER45>', 'ANNOUNCER': '<PLAYER46>', 'CHRIS PRAMAS': '<PLAYER47>', 'IPHONE': '<PLAYER48>', 'BRIAN': '<PLAYER49>', 'MARK': '<PLAYER50>', 'DENISE': '<PLAYER51>', 'AMANDINE': '<PLAYER52>', 'NOELLE': '<PLAYER53>', 'JON': '<PLAYER54>', 'CHRIS WILLMOTT': '<PLAYER55>', 'JOE': '<PLAYER56>', 'KHARY': '<PLAYER57>', 'JOEL': '<PLAYER58>', 'ALL - SAM': '<PLAYER59>', 'TOY IN BAG': '<PLAYER60>', 'BABS': '<PLAYER61>', 'SUMALEE': '<PLAYER62>', 'MAX': '<PLAYER63>', 'CAST': '<PLAYER64>', 'AMY': '<PLAYER65>', 'ERIC': '<PLAYER66>', 'CAMERAMAN': '<PLAYER67>', 'YURI': '<PLAYER68>', 'YASMINE': '<PLAYER69>', 'SEAN': '<PLAYER70>', 'ALLISON': '<PLAYER71>', 'ANNET': '<PLAYER72>', 'JOEY': '<PLAYER73>', 'LENORE': '<PLAYER74>', 'DEBORAH': '<PLAYER75>'}

In [23]:
def map_actor_to_id(actor):
    # manual some
    if actor == 'Will Friedle':
        return '<PLAYER30>'

    for key, value in actordict.items():
        if key.lower() in actor.lower():
            return value
    return actor

c1_main_df['actor_id'] = c1_main_df['Actor'].apply(map_actor_to_id)
c1_guests_df['actor_id'] = c1_guests_df['Actor'].apply(map_actor_to_id)

In [24]:
display(c1_main_df)
display(c1_guests_df)

,Actor,Character,Race,Class,actor_id
0,Orion Acaba(episode 1-27),Tiberius Stormwind,Dragonborn,Sorcerer (Draconic Bloodline),<PLAYER5>
1,Laura Bailey,Vex'ahlia,Half-elf,Ranger (Beast Master)/Rogue (Assassin),<PLAYER7>
2,Taliesin Jaffe,Percival de Rolo,Human,Fighter (Gunslinger),<PLAYER3>
3,Ashley Johnson,Pike Trickfoot,Gnome,Cleric (War Domain),<PLAYER13>
4,Liam O'Brien,Vax'ildan,Half-elf,Rogue (Assassin)/Paladin (Oath of Vengeance)/Druid,<PLAYER6>
5,Liam O'Brien,Lieve'tel Toluse,Elf,Cleric (Grave Domain),<PLAYER6>
6,Liam O'Brien,Derrig,Half-elf,Fighter (Battle Master),<PLAYER6>
7,Marisha Ray,Keyleth,Half-elf,Druid (Circle of the Moon),<PLAYER2>
8,Sam Riegel,Scanlan Shorthalt,Gnome,Bard (College of Lore),<PLAYER4>
9,Sam Riegel,Taryon Darrington,Human,Artificer (Battle Smith; formerly Alchemist),<PLAYER4>


,Actor,Character,Race,Class,actor_id
0,Mary Elizabeth McGlynn,Zahra Hydris,Tiefling (Bloodline of Asmodeus),"Warlock (The Great Old One, Pact of the Tome)",<PLAYER27>
1,Felicia Day,Lyra,Human,Wizard (School of Transmutation),<PLAYER28>
2,Will Friedle,Kashaw Vesh,Human,Cleric (Life Domain),<PLAYER30>
3,Wil Wheaton,Thorbir Falbek,Dwarf,Fighter (Battle Master),<PLAYER14>
4,Kit Buss,Lillith Daturai,Tiefling (Bloodline of Asmodeus),Wizard (School of Illusion),<PLAYER33>
5,Jason Charles Miller,Garthok,Half-orc,Rogue (Thief),<PLAYER38>
6,Chris Hardwick,Gern Blanston,Dragonborn (Ravenite),Wizard (School of Necromancy),<PLAYER39>
7,Chris Perkins,Shale,Goliath,Fighter (Battle Master),<PLAYER37>
8,Patrick Rothfuss,Kerrek,Human,Paladin (Oath of Devotion),<PLAYER40>
9,ND Stevenson,Tova,Dwarf (Werebear),Blood hunter (Order of the Lycan),<PLAYER31>


In [25]:
c1_actors_df = pd.concat([c1_main_df, c1_guests_df], ignore_index=True)
c1_actors_df = c1_actors_df.rename(columns={'Actor': 'actor', 'Character': 'character', 'Character': 'character', 'Class':'class'})

# c1_main_df.to_csv('campaign_1_actors.csv', index=False)


In [26]:
c1_actors_df.to_csv('campaign_1_actors.csv', index=False)

### Magic Item -> Inventory


In [27]:
c1_items_url = 'https://criticalrole.fandom.com/wiki/Category:Magic_Items#Campaign_One'

c1_items_page = requests.get(c1_items_url)

c1_items_soup = BeautifulSoup(c1_items_page.content, 'html.parser')

c1_items_table = c1_items_soup.find('table')

div_items = c1_items_table.find_all('div', class_='wds-tab__content')

items_rows = [tr for tr in div_items[0].find_all('tr') if tr.get('style') != 'height:2px']

def parse_items_rows(rows):
    item_cols = ['item_type', 'item_name']
    item_cells = []
    for row in rows:
        item_name = row.find('b').text.strip()
        # items = [i.text.strip() for i in row.find_all('a')]
        for i in row.find_all('a'):
            item_cells.append([item_name ,i.text.strip()])
    return item_cols, item_cells 

In [28]:
item_cols, items_rows = parse_items_rows(items_rows)
items_df = pd.DataFrame(items_rows, columns=item_cols)

In [29]:
items_df.to_csv('campaign_1_items.csv', index=False)

# Extract the D&D game state

Fields:
- Character Name (/)
- Race (/)
- Class (/)
- Pronouns ()
- Inventory (/)
- In combat? (true/false) ()
- In character? (true/false) ()
- Action ()

## Inventory ?

In [30]:
def check_invent(utterance):
    for item in items_df['item_name'].tolist():
        if item.lower() in utterance.lower():
            return item
    return 'None'

tqdm.pandas(desc="Processing")
df['item'] = df['utterance'].progress_apply(check_invent)

Processing: 100%|██████████| 398682/398682 [00:07<00:00, 55072.91it/s]


In [31]:
# df_with_item = df[df['item'] not in ['None']]
df_with_item = df[(df['item'] != 'None') & (df['item'] != 'Whisper')]
display(df_with_item.loc[40000:50000])

,fileID,number,speaker,utterance,item
40342,C1E016,973,<DM>,"He gives a look to the rest of the group here, and you can see there's a noticeable bead of sweat he kind of wipes off his forehead. ""I just needed to confirm that this was not some haphazardly constructed fake or some other artifact mistaken for such a horn, but we indeed have within our presence one of the Horns of Orcus, and this does not bode well.""",Horns of Orcus
40713,C1E016,1344,<PLAYER6>,Everybody stand back. I'm going to click my Boots of Haste--,Boots of Haste
40715,C1E016,1346,<PLAYER6>,"Yeah, me. I'm going to click my Boots of Haste, I'm going to run, I'm going to do a sort of a running roll, land on the closest circle and wait for half a tick and then roll off of it.",Boots of Haste
40719,C1E016,1350,<DM>,"So as you guys are about to go, ""Wait!"", Vax is already off. Clicks the boots. The arcane magic of the Boots of Haste kick into his body. He leaps forward. Make an acrobatics check.",Boots of Haste
41322,C1E016,1953,<PLAYER6>,Then the keen dagger follows. That is a 27.,Keen Dagger
41326,C1E016,1957,<PLAYER6>,And then the flametongue dagger.,Flametongue Dagger
41334,C1E016,1965,<PLAYER5>,Cloak of Displacement.,Cloak of Displacement
41690,C1E016,2321,<PLAYER5>,"I'd like to walk up as I see this going on. I'm going to take out my deck of illusions. I'm going to toss two cards both ways, two cards on each side. So I'm going to do this, real quick.",Deck of Illusions
41693,C1E016,2324,<PLAYER5>,"Deck of illusions, everyone.",Deck of Illusions
42351,C1E017,266,<PLAYER6>,"I don't start running, but I'm following behind him and I take out my Keen Dagger and just start sort of dancing it across my hand as I follow after him.",Keen Dagger


In [32]:
def loc_window(df, idx, size):
    return df.loc[idx-size:idx+size+1]

In [33]:
loc_window(df, 49081, 7)


,fileID,number,speaker,utterance,item
49074,C1E020,495,<DM>,"We'll get to that. As you lean forward, forcing your presence on this Bastion, his hand's still holding the edge of his rapier. He steps forward into you, to the point where you're almost forehead to forehead. ""Sorry, I have to lean down to reach your level. But if you're going to go about asking questions and not offering proper compensation for information, well, then. I think we have no business here.""",None
49075,C1E020,496,<PLAYER14>,"If it's compensation you're looking for, let's just speak plain. And I pull out four gold.",None
49076,C1E020,497,<PLAYER6>,"And while he has their attention, I'm going to check out his pockets.",None
49077,C1E020,498,<DM>,Go ahead and make a perception check.,None
49078,C1E020,499,<PLAYER6>,"Ew, that's not good. I'm going to use Luck. That's better. That's all right. Perception, you said? 28.",None
49079,C1E020,500,<DM>,"28? Taking a moment as you sneak up behind the one he's talking to-- the two guys are shoulder to shoulder, squaring off with the rest of this group. The one that had been relieving himself in the alleyway is a little bit closer to you. You can see as he put his tunic right and everything, he jostled around and revealed a couple of small pouches on his side. One of which, you heard a clinking of glass on the inside. Go ahead and make a sleight-of-hand check.",None
49080,C1E020,501,<PLAYER2>,"Go rogue, go.",None
49081,C1E020,502,<PLAYER6>,"Because I'm wearing a Cloak of Elvenkind, which I know affects stealth checks but not--",Cloak of Elvenkind
49082,C1E020,503,<DM>,"Not sleight of hand, no.",None
49083,C1E020,504,<PLAYER6>,That's all right. 20.,None


### Try LM predicting Items
- later



In [34]:
import spacy
import re
from collections import deque


In [35]:

nlp = spacy.load("en_core_web_sm")

verbs = {
    "wear", "use", "click", "put", "pull", "draw", "activate", "trigger",
    "equip", "hold", "cast", "grab", "place", "unsheath", "brandish"
}
actions = [
    "put on", "pull out", "click the", "draw my", "draw the", "activate the",
    "equip the", "cast with", "hold up", "use my", "drop a bead", "unsheath"
]

owner_history = {}

mentioned = deque(maxlen=20)

def match_action_phrase(text, item_name):
    text = text.lower()
    item = item_name.lower()
    return any(phrase in text and item in text for phrase in actions)

def owner_fin(owner):
    if owner == "<DM>":
        return "Unknown"
    return owner

def detect_owner(entry):
    speaker = entry["speaker"]
    utt = entry["utterance"]
    item_name = entry["item"]
    item_lower = item_name.lower()

    if item_lower in owner_history:
        mentioned.append((item_lower, owner_history[item_lower]))

    doc = nlp(utt)

    # subjects
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ in verbs:
                subject = [w for w in token.lefts if w.dep_ in ("nsubj", "nsubjpass")]
                if subject and subject[0].text.lower() == "i" and item_lower in sent.text.lower():
                    owner_history[item_lower] = speaker
                    mentioned.append((item_lower, speaker))
                    return owner_fin(speaker)

    # actions
    if match_action_phrase(utt, item_name):
        if re.search(r"\b(I|my|me|I'm|I've)\b", utt, re.IGNORECASE):
            owner_history[item_lower] = speaker
            mentioned.append((item_lower, speaker))
            return owner_fin(speaker)

    # referring
    if re.search(r"\bit\b|\bthe\b.*{}".format(re.escape(item_lower.split()[-1])), utt.lower()):
        for ref_item, ref_owner in reversed(mentioned):
            if item_lower.startswith(ref_item) or ref_item.startswith(item_lower):
                return owner_fin(ref_owner)

    # regex pronouns (in original paper)
    if item_lower in utt.lower():
        if re.search(r"\b(I|my|me|I'm|I've)\b", utt, re.IGNORECASE):
            owner_history[item_lower] = speaker
            mentioned.append((item_lower, speaker))
            return owner_fin(speaker)

    return "Unknown"




In [36]:
sample = [
    {
        "speaker": "PLAYER6",
        "utterance": "The one that's occupied with Grog, I'm going to jump on it and cling my legs to the side and try to slice down as I go, over the side wall. Okay. First is a 28. I'll just do that. And it's a sneak attack because it's occupied with Grog, obviously. So that is a two-- this is my Keen Dagger. That's a nine plus... nine, 13, 16, 19, 24. Nine plus 24 is 33, and then the second dagger, Flametongue, nine, ten, 19.",
        "item": "occupied"
    },
]

for entry in sample:
    owner = detect_owner(entry)
    print(f"{entry['item']} → Owner: {owner}")

occupied → Owner: PLAYER6


In [37]:
tqdm.pandas(desc="Detecting owner")
def detect_owner_(row):
    if row['item'] not in ['None', 'Whisper']:
        return detect_owner(row)
    return 'None'
df['owner'] = df.progress_apply(detect_owner_, axis=1)

Detecting owner: 100%|██████████| 398682/398682 [00:11<00:00, 36108.02it/s] 


In [38]:
df_with_item = df[(df['item'] != 'None') & (df['item'] != 'Whisper')]
display(df_with_item.loc[30000:31000])

,fileID,number,speaker,utterance,item,owner
30094,C1E011,2533,<PLAYER5>,"And, the Endless Rope? As I'm asking for that, I take out my Immovable Rod and I'm going to place it, kind of, in an exit position and (clicks)! Make it stay right above there.",Immovable rod,<PLAYER2>
30249,C1E011,2688,<PLAYER6>,I am clicking my Boots of Haste.,Boots of Haste,<PLAYER6>
30380,C1E011,2819,<PLAYER7>,"Okay. The secondary attack I wanted to shoot through my Bloodseeking Bow, though.",Bloodseeking Bow,<PLAYER7>
30399,C1E011,2838,<PLAYER7>,And then the Bloodseeking Bow bleeds him.,Bloodseeking Bow,<PLAYER7>
30687,C1E011,3126,<DM>,"There is one rope, currently, that is attached to the Immovable Rod, yes.",Immovable rod,<PLAYER2>
30709,C1E011,3148,<PLAYER6>,"So there's no plus-- I forget what's added for, like, my dexterity and what's the magic-- The keen-- Oh, it's two for the keen dagger, so one plus five is six. Now here comes the Sneak Attack damage. Six plus-- There's nine, 12, 13, 18, 22 for the first. Oh, and Natural 20 is a crit.",Keen Dagger,<PLAYER6>
30778,C1E011,3217,<DM>,"15? You try and move out of the way but the beam hits you on the side of your torso. You feel a familiar sensation, back when that proto-lich was drawing the life force from you from the phylactery that was sewn in your chest. I need you-- Nope. You take 55 points of damage.",Phylactery,Unknown


## Spell things?
- https://criticalrole.fandom.com/wiki/Category:Spells 
might be useful for spell checking

## In Combat ?
- true/false

In [39]:
# check for dice rolls

# def check_dice_roll(utterance):
#     dice_roll_regex = r"\b\d{1,2}d\d{1,2}\b"
#     if re.search(dice_roll_regex, utterance):
#         return True
#     return False

df[df['utterance'].str.contains(r"\b\d{1,2}d\d{1,2}\b")]
df[df['utterance'].str.contains(r"\b(\d{0,3})\s*[dD]\s*\d{1,3}(\s*[+-]\s*\d{1,3})?\b")]

# df[df['utterance'].str.contains(r"\b\d{1,2}\b")]
df[df['utterance'].str.contains(r"roll")]

C:\Users\sonso\AppData\Local\Temp\ipykernel_20616\3093205834.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['utterance'].str.contains(r"\b(\d{0,3})\s*[dD]\s*\d{1,3}(\s*[+-]\s*\d{1,3})?\b")]


,fileID,number,speaker,utterance,item,owner
1,C1E001,1,<DM>,"Welcome to first episode of Critical Role, and what this basically is is a continuation of our weekly D&D game. Me and a bunch of other likely nerdy and enjoyable voice actors gathering around, rolling some dice, killing some creatures, having some adventure. Now we have the pleasure of bringing it on the stream for you to watch, enjoy, and occasionally interact with. Before we get to that, to give you a little backstory on the characters you'll be seeing this evening, we're going to play some videos for you in a second. Do note for all you hardcore gamers out there, a lot of this is house-ruled, loosey-goosey having a good time. So all you number crunchers, stop paying attention there, just have fun with it. Nevertheless, we have some background story on many of the characters you'll be seeing this evening to help you jump into the story. Let's go ahead and enjoy those in a minute, then we'll introduce the players. So have fun!",None,None
7,C1E001,7,<PLAYER4>,"Oh, you haven't heard of Scanlan Shorthalt? Well, gird your loins, ladies, because he has his eye on you. A talented musician, master of disguise, and dashingly handsome in his own mind, Scanlan sings songs almost as much as he sings his own praises. Born a poor gnome, Scanlan used his endless charm and soaring tenor voice to croon for coin and support his single mother. One day, he was discovered by a half-orc promoter, and joined Dr. Dranzel's Spectacular Traveling Troupe where he learned the ways of the world, and honed his skills as a bard extraordinaire. A loner much of his life, Scanlan has never quite come to terms with the violent death of his mother at the hands of a goblin invasion. While his years on the road provided many, shall we say, educational experiences with the opposite sex, deep down Scanlan yearns for the one thing he's never known: the true love of a fellow gnome. Still, Scanlan considers himself a lover first, performer second, and fighter distant third. On the battlefield, he'll support his allies, but rarely draws blood, unless it's to protect fellow gnome, Pike. Count on Scanlan for a hearty laugh, a rollicking song, and a twinkle in his eye that melts hearts and makes the females swoon.",None,None
53,C1E001,53,<DM>,"Yep, the barbarian for his first and only time so far managed to avert battle through a social encounter and rolled pretty damn well on a persuasion check. So you got one. You get one. You got your one. Never again.",None,None
287,C1E001,287,<PLAYER6>,First die rolling. Nice!,None,None
289,C1E001,289,<DM>,"This would be a straight intelligence roll on this one, so your general knowledge of dwarves.",None,None
...,...,...,...,...,...,...
398392,C2E046,2103,<DM>,"Pretty sure as a hexblade warlock you get to add your charisma bonus to attack and damage rolls, right? Am I wrong about that?",None,None
398493,C2E046,2204,<DM>,"All right. It'll take you a little bit, but go ahead and roll a stealth check.",None,None
398568,C2E046,2279,<PLAYER2>,Go ahead. You might roll higher than I did.,None,None
398569,C2E046,2280,<PLAYER13>,What would the roll be? Is it perception?,None,None


In [40]:
loc_window(df,398568, 3)

,fileID,number,speaker,utterance,item,owner
398565,C2E046,2276,<DM>,Sure.,None,None
398566,C2E046,2277,<PLAYER2>,"I do, too.",None,None
398567,C2E046,2278,<PLAYER13>,"Oh, you already have it?",None,None
398568,C2E046,2279,<PLAYER2>,Go ahead. You might roll higher than I did.,None,None
398569,C2E046,2280,<PLAYER13>,What would the roll be? Is it perception?,None,None
398570,C2E046,2281,<DM>,Mm-hmm.,None,None
398571,C2E046,2282,<PLAYER13>,16.,None,None
398572,C2E046,2283,<DM>,You make out the same thing.,None,None


In [41]:
combat_state = False
last_atk_count = 0

persistent = 15

DICE_REGEX = r"\b\d{0,2}\s*[dD]\s*\d{1,3}(\s*[+-]\s*\d{1,3})?\b"

attacks = [
    "attack", "hit", 
    "damage", "strike",
    "swing", "slash", 
    "stab", "cast", "shoot",
]

def detect_combat_state(entry):
    global combat_state, last_atk_count

    utt = entry["utterance"]
    speaker = entry["speaker"]
    doc = nlp(utt.lower())

    # roll initiative
    if re.search(r"\b(?:roll(ing)? )?initiative\b", utt, re.IGNORECASE):
        combat_state = True
        last_atk_count = 0
        return True

    # if re.search(r"\b(attack|hit|damage|strike|swing|slash|stab|cast|shoot)\b", utt, re.IGNORECASE):
    #     if "roll" in utt or any(tok.lemma_ == "roll" for tok in doc):
    #         combat_state = True
    #         last_atk_count = 0
    #         return True

    # assume combat starts with init roll only (for now)
    # CASE1: attack words
    if any(tok.lemma_ in attacks for tok in doc):
        combat_state = True
        last_atk_count = 0
        return True

    # CASE2: dice rolls
    if re.search(DICE_REGEX, utt, re.IGNORECASE):
        combat_state = True
        last_atk_count = 0
        return True

    # no attack detected
    if combat_state:
        last_atk_count += 1
        if last_atk_count >= persistent:
            combat_state = False
            return False
        else:
            return True

    return False


In [42]:
# df_w_combat = df.copy()

In [43]:
tqdm.pandas(desc="Detecting Combat State")
def detect_owner_(row):
    return detect_combat_state(row)
df['in_combat'] = df.progress_apply(detect_combat_state, axis=1)


Detecting Combat State:   2%|▏         | 8143/398682 [00:28<22:25, 290.33it/s]


KeyboardInterrupt: 

In [ ]:
# save to csv
df.to_csv('df_with_combat_v1.csv', index=False)

In [ ]:
df[df['in_combat'] == True]

,fileID,number,speaker,utterance,item,owner,in_combat
1,C1E001,1,<DM>,"Welcome to first episode of Critical Role, and what this basically is is a continuation of our weekly D&D game. Me and a bunch of other likely nerdy and enjoyable voice actors gathering around, rolling some dice, killing some creatures, having some adventure. Now we have the pleasure of bringing it on the stream for you to watch, enjoy, and occasionally interact with. Before we get to that, to give you a little backstory on the characters you'll be seeing this evening, we're going to play some videos for you in a second. Do note for all you hardcore gamers out there, a lot of this is house-ruled, loosey-goosey having a good time. So all you number crunchers, stop paying attention there, just have fun with it. Nevertheless, we have some background story on many of the characters you'll be seeing this evening to help you jump into the story. Let's go ahead and enjoy those in a minute, then we'll introduce the players. So have fun!",None,None,True
2,C1E001,2,<PLAYER1>,"Right, listen up! If you have ale, then you have a friend in Grog Strongjaw! A goliath of towering height and size, this barbarian has an appetite for the two great loves in his life: combat, women, and ale! [record scratch]",None,None,True
3,C1E001,3,<PLAYER1>,"Wait. Easily the brains of the group, Grog is often consulted for his vast knowledge of shapes, colors, and shiny things! Also ale. In his early years, armed with his two-handed greataxe, Grog often enjoyed proving his might amongst the ranks of his family's wandering herd. But after coming upon an unsuspecting elderly gnome in the woods, he objected to the killing of such an innocent life. A creature of impulse, Grog felt only pity for this-- well, this terrified little thing. And his disobedience cost him dearly. Beaten bloody, and banished by the herd leader, his Uncle Kevdak, Grog was abandoned and left to die. Exiled from his herd, it was then that the relative of the very gnome he fought to save, saved him. It was the kindness of a gnome cleric named Pike that healed Grog, bringing him back from death's edge. And they have remained close friends ever since. Most nights, Grog can be found challenging entire taverns to wrestling matches! Or accompanying Scanlan to the nearest house where you pay for lady favors. Also ale!",None,None,True
4,C1E001,4,<PLAYER2>,"A first impression of Keyleth would leave you with little information on the half-elven druid. You might even think that her social awkwardness due to her sheltered upbringing is kind of sweet. Of course, it would be unwise underestimate her based on first impressions. [thunder crack]",None,None,True
5,C1E001,5,<PLAYER2>,"Under that un-intimidating petite frame is a vicious beast waiting to be unleashed, whose natural powers have made even the fiercest of champions pee their pants, literally! Born to the Air Tribe of the Ashari people, Keyleth was raised with a deep love of nature and the elemental magics. It is her people's inherent duty to protect the delicate areas in Tal'Dorei where the four elemental planes begin to bleed with this realm. Since she was a little girl, she had quite a knack for air manipulation and beast shaping abilities. Well, if you consider kittens and flying squirrels to be little beasts. Which, I do. Anyways, it wasn't long before the headmaster of the tribe, her father, Korren, realized her true prodigious abilities and she was inveterated to succeed him as the next headmaster. Just like that, her jovial childhood was stripped and replaced with endless spell memorization, teachings from ancient traditions, and exceedingly high expectations. Every druid leader-to-be must embark on a journey to seek out the sister tribes in order to introduce and establish respect amongst the fellow headmasters. They call this the Aramente, or Noble Odyssey. When her father felt she was ready, he set her on the path to truly discovering herself, not knowing when, or if, 

In [ ]:
loc_window(df, 395827, 25)

,fileID,number,speaker,utterance,item,owner,in_combat
395802,C2E045,3229,<DM>,You're like-- (groaning). Both roll initiative to see where you fall on the initiative order.,None,None,True
395803,C2E045,3230,<PLAYER4>,"Mmm, eight.",None,None,True
395804,C2E045,3231,<PLAYER6>,Seven.,None,None,True
395805,C2E045,3232,<DM>,Eight? Okay.,None,None,True
395806,C2E045,3233,<PLAYER4>,Caleb! What's going on?,None,None,True
395807,C2E045,3234,<PLAYER7>,You don't see anybody in there.,None,None,True
395808,C2E045,3235,<PLAYER1>,"Oh yeah, there's nobody in there!",None,None,True
395809,C2E045,3236,<DM>,"You're in a chamber. You land there and you see this angry, burning dragon and this tiny little version of the mage that you walked through the den of and that's it. You see the glowing sphere here on the pedestal, these lines of energy that meet in the center. The fire elemental in the back way, burning. You're Troy in Community walking in with the pizzas.",None,None,True
395810,C2E045,3237,<PLAYER6>,"Minor question. Do I smell burnt ozone, since you said that just a second ago?",None,None,True
395811,C2E045,3238,<DM>,You do smell the electrical remainder in the air.,None,None,True


#### Combat right now is not so good , will try other method 


## OOC vs IC